In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import os, glob, inspect, sys
import re

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import epri_mc_lib_2 as mc
from importlib import reload
reload(mc)


<module 'epri_mc_lib_2' from '/home/nick/Bureau/Programming/S2DS/Oct20_EPRI/Task2/NB/epri_mc_lib_2.py'>

### Import data and merge replicates

In [65]:
merged = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_FT_TEP_UT_on_ID.csv'),
                    index_col=0)
merged.index = merged.index.str.rstrip('-12345')
mean_df = merged.groupby('ID').mean()
# Get rid of cold work
cw_regex = re.compile("[0-9]+$") 
mean_df['cold_work'] = [int(re.search(cw_regex,x).group()) for x in mean_df.index]
mean_df.index = mean_df.index.str.rstrip('02468')
mean_df.index = mean_df.index.str.rstrip('-')

## Select columns of interest

In [70]:
mean_df = mean_df[mc.regression_cols].dropna()


,KJIC,MS_Avg,TEP_average,Beta_avg,PC_IF_2.25MHz,PC_IF_3.5MHz,PC_BS,cold_work
ID,,,,,,,,
304,162.000000,0.12000,-1.54700,0.002743,0.0,0.0,0.0,0
304,105.780000,0.49600,-1.32200,0.002093,50.0,59.0,-12.0,20
304,100.700000,1.35000,-1.21600,0.001941,57.0,79.0,-20.0,40
304,91.320000,1.37100,-1.24900,0.002567,98.0,139.0,3.0,60
304,73.775000,3.96600,-0.97600,0.002420,94.0,149.0,10.0,80
316,111.240000,0.03900,-1.50600,0.002433,0.0,0.0,0.0,0
316,69.960000,0.03600,-1.57300,0.002785,42.0,37.0,30.0,20
316,68.220000,0.04400,-1.64000,0.002494,70.0,60.0,-34.0,40
316,69.000000,0.04925,-1.67375,0.002292,117.0,200.0,-57.0,60


## Scale data

In [73]:
scaled_df = mc.scale_general(mean_df, MinMaxScaler())
scaled_df

(          KJIC    MS_Avg  TEP_average  Beta_avg  PC_IF_2.25MHz  PC_IF_3.5MHz  \
 ID                                                                             
 304   0.758811  0.017354     0.148811  0.958190       0.000000      0.000000   
 304   0.387313  0.076272     0.412973  0.318489       0.083195      0.051937   
 304   0.353744  0.210091     0.537423  0.168552       0.094842      0.069542   
 304   0.291762  0.213382     0.498679  0.785035       0.163062      0.122359   
 304   0.175826  0.620010     0.819196  0.640251       0.156406      0.131162   
 316   0.423392  0.004662     0.196947  0.653342       0.000000      0.000000   
 316   0.150617  0.004192     0.118286  1.000000       0.069884      0.032570   
 316   0.139119  0.005445     0.039624  0.713502       0.116473      0.052817   
 316   0.144273  0.006268     0.000000  0.514357       0.194676      0.176056   
 316   0.080837  0.037255     0.033754  0.771024       0.188020      0.115317   
 347   1.000000  0.001998   